# Lab | Data Structuring and Combining Data

## Challenge 1: Combining & Cleaning Data

In this challenge, we will be working with the customer data from an insurance company, as we did in the two previous labs. The data can be found here:
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv

But this time, we got new data, which can be found in the following 2 CSV files located at the links below.

- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv

Note that you'll need to clean and format the new data.

Observation:
- One option is to first combine the three datasets and then apply the cleaning function to the new combined dataset
- Another option would be to read the clean file you saved in the previous lab, and just clean the two new files and concatenate the three clean datasets

In [2]:
#Cleaning data from file1.csv:

import pandas as pd
import numpy as np

df_file1=pd.read_csv('https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv')
df_file2=pd.read_csv('https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv')
df_file3=pd.read_csv('https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv')

df=pd.concat([df_file1, df_file2, df_file3], axis=0, join="outer")

df.columns = df.columns.str.strip().str.lower().str.replace(r'\s+', '_', regex=True)

#If you apply rename you have to deal with duplicated columns again. So is better to do it the last thing.
# 1. Normalize and rename columns
# 2. Deal with duplicated columns
# 3. Clean data

df=df.rename(columns={'st':'state'})
df = df.loc[:, ~df.columns.duplicated(keep='first')].copy()

df['gender'] = (
    df['gender']
      .astype('string') 
      .str.strip()
      .str.lower()
      .map({'m':'M','male':'M','masculino':'M',
            'f':'F','female':'F','femal':'F','femenino':'F'})
      .fillna('U')  
)

df['state']=df['state'].str.lower().replace({'cali':'California','az':'Arizona','wa':'Washington'}).fillna('Other').str.title()

df['education']=df['education'].str.lower().str.replace('bachelors','bachelor').str.title() 

df['customer_lifetime_value']=df['customer_lifetime_value'].str.replace('%',' ').str.strip()

df['vehicle_class']=df['vehicle_class'].str.lower().replace({'sports car':'Luxury','luxury suv':'Luxury','luxury car':'Luxury'}).str.title()

df['customer_lifetime_value'] = pd.to_numeric(df['customer_lifetime_value'], errors='coerce')

df['number_of_open_complaints'] = (
    pd.to_numeric(df['number_of_open_complaints'].astype('string').str.split('/').str[1], errors='coerce')
    .astype('Int64') 
)

# Handling missing values.
null_counts = df.isna().sum()
for n_nulls in null_counts.unique():    # Iterate over unique counts of nulls
      cols_group = null_counts[null_counts == n_nulls].index  # Get columns with that count
      df.dropna(subset=cols_group, inplace=True)

df = df.drop_duplicates()

df = df.reset_index(drop=True)

df

,customer,state,gender,education,customer_lifetime_value,income,monthly_premium_auto,number_of_open_complaints,policy_type,vehicle_class,total_claim_amount
0,QZ44356,Arizona,F,Bachelor,697953.59,0.0,94.0,0,Personal Auto,Four-Door Car,1131.464935
1,AI49188,Nevada,F,Bachelor,1288743.17,48767.0,108.0,0,Personal Auto,Two-Door Car,566.472247
2,WW63253,California,M,Bachelor,764586.18,0.0,106.0,0,Corporate Auto,Suv,529.881344
3,GA49547,Washington,M,High School Or Below,536307.65,36357.0,68.0,0,Personal Auto,Four-Door Car,17.269323
4,OC83172,Oregon,F,Bachelor,825629.78,62902.0,69.0,0,Personal Auto,Two-Door Car,159.383042
...,...,...,...,...,...,...,...,...,...,...,...
2052,HV85198,Arizona,M,Master,847141.75,63513.0,70.0,0,Personal Auto,Four-Door Car,185.667213
2053,BS91566,Arizona,F,College,543121.91,58161.0,68.0,0,Corporate Auto,Four-Door Car,140.747286
2054,IL40123,Nevada,F,College,568964.41,83640.0,70.0,0,Corporate Auto,Two-Door Car,471.050488
2055,MY32149,California,F,Master,368672.38,0.0,96.0,0,Personal Auto,Two-Door Car,28.460568


# Challenge 2: Structuring Data

In this challenge, we will continue to work with customer data from an insurance company, but we will use a dataset with more columns, called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by performing data cleaning, formatting, and structuring.

In [3]:
df_marketing=pd.read_csv('https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv')
pivot_df_marketing=df_marketing.pivot_table(index='sales_channel', columns='response',aggfunc='size', fill_value=0)
pivot_df_marketing



response,No,Yes
sales_channel,,
Agent,3379,742
Branch,2696,326
Call Center,1920,221
Web,1449,177


In [6]:
#To know wich sales_channel has the highest response we use this pivot table:
#pivot_df_lifetime_value=df_marketing.pivot_table(index='gender', columns='education', values='customer_lifetime_value', aggfunc='mean', fill_value=0)
#pivot_df_lifetime_value

pivot_df_marketing=df_marketing.pivot_table(values='total_claim_amount', index='sales_channel', aggfunc='sum').round(2)
pivot_df_marketing



,total_claim_amount
sales_channel,
Agent,1810226.82
Branch,1301204.00
Call Center,926600.82
Web,706600.04


1. You work at the marketing department and you want to know which sales channel brought the most sales in terms of total revenue. Using pivot, create a summary table showing the total revenue for each sales channel (branch, call center, web, and mail).
Round the total revenue to 2 decimal points.  Analyze the resulting table to draw insights.

2. Create a pivot table that shows the average customer lifetime value per gender and education level. Analyze the resulting table to draw insights.

## Bonus

You work at the customer service department and you want to know which months had the highest number of complaints by policy type category. Create a summary table showing the number of complaints by policy type and month.
Show it in a long format table.

*In data analysis, a long format table is a way of structuring data in which each observation or measurement is stored in a separate row of the table. The key characteristic of a long format table is that each column represents a single variable, and each row represents a single observation of that variable.*

*More information about long and wide format tables here: https://www.statology.org/long-vs-wide-data/*

In [5]:
# Your code goes here